In [4]:
!pip install tensorflow requests

In [16]:
dataset = '/bohr/ai4spulseeis-lr97/v5'

import pickle
from os import listdir, remove
from os.path import exists, join, splitext
import numpy as np
import tensorflow as tf
from requests import get
from tqdm import tqdm
from hashlib import md5

if exists('predictor.h5'): remove('predictor.h5')
response = get('https://raw.githubusercontent.com/breadbread1984/EIS_prediction/main/predictor.h5')
with open('ckpt.tar.gz', 'wb') as f:
  f.write(response.content)
assert 'cadc752fceef46827d3b9f2ca81c9f7e' == md5(response.content).hexdigest()
if exists('sos.npy'): remove('sos.npy')
response = get('https://raw.githubusercontent.com/breadbread1984/EIS_prediction/main/sos.npy')
with open('sos.npy', 'wb') as f:
  f.write(response.content)
assert '7d4b3fe44708d9de47a30b83ffe86da0' == md5(response.content).hexdigest()

In [10]:
def Trainer(hidden_dim = 256, layers = 1):
  pulse = tf.keras.Input((None, 2))
  eis = tf.keras.Input((None, 2))

  pulse_embed = tf.keras.layers.Dense(hidden_dim)(pulse)
  eis_embed = tf.keras.layers.Dense(hidden_dim)(eis)

  rnn = tf.keras.layers.RNN([tf.keras.layers.LSTMCell(hidden_dim) for i in range(layers)], return_sequences = True, return_state = True)
  results = rnn(pulse_embed)
  state = results[1:]
  results = rnn(eis_embed, initial_state = state)
  hidden = results[0]

  eis_update = tf.keras.layers.Dense(2)(hidden)

  return tf.keras.Model(inputs = (pulse, eis), outputs = eis_update)


In [ ]:
trainer = Trainer()
trainer.load_weights('predictor.h5')
sos = tf.constant(np.load('sos.npy'))

output = open('submission.csv','w')
output.write('test_data_number,SOC(%),EIS_real,EIS_imaginary\n')
for f in listdir(join(dataset, 'test_datasets')):
  stem, ext = splitext(f)
  if ext != '.pkl': continue
  test_num = int(stem.replace('test_pulse_', ''))
  with open(join(dataset, 'test_datasets', f), 'rb') as f:
    data = pickle.load(f)
  for SOC, pulse_samples in tqdm(data.items()):
    soc = SOC.replace('%SOC','')
    pulse = tf.expand_dims(tf.stack([pulse_samples['Voltage'], pulse_samples['Current']], axis = -1), axis = 0) # pulse.shape = (1, seq, 2)
    eis = tf.tile(sos, (pulse.shape[0], 1, 1))
    for i in range(51):
      pred = trainer([pulse, eis])
      eis = tf.concat([eis, pred[:,-1:,:]], axis = -2)
    eis = eis[:,1:,:][0]
    for e in eis:
      output.write(','.join([str(test_num),soc,str(e[0].numpy().item()),str(e[1].numpy().item())]) + '\n')
output.close()


 76%|█████████████████████████████████████████████████████████▍                  | 37/49 [01:37<00:31,  2.59s/it]